<a href="https://colab.research.google.com/github/Alaaibrahim2/Alaa/blob/main/text_summarization_by_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("/content/labeled.csv")

In [2]:
df

,example_id,paragraph,summary,min_len,max_len
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...,631,681
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...,365,415
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...,486,536
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...,684,734
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...,361,411
...,...,...,...,...,...
149,149,حزب الوفد سيحتفل بمئوية ثورة 1919 يوم 9 مارس ا...,احتفال مئوية ثورة 1919 كان من منطلق وطني ليس ح...,518,568
150,150,حيث أعلن مجلس قيادة الثورة في 18 يونيه 1953 قي...,مجلس قيادة الثورة أعلن عن قيام الجمهورية المصر...,397,447
151,151,وبرغم أن عبد الرحمن فهمي كان يضم في ذلك الجهاز...,ضم عبد الرحمن فهمي في الجهاز السري عدد كبير من...,405,455
152,152,ولم تقتصر مقومات بورسعيد كمدينة عالمية منذ نشأ...,امتدت بورسعيد لكي تشمل الطابع الثقافي للمدينة،...,484,534


In [5]:
import re
!pip install pyarabic
!pip install transformers
from pyarabic.araby import strip_tashkeel, strip_tatweel
from transformers import AutoTokenizer

def preprocess_arabic_text(text):
    # Remove stop words
    stop_words = ["و", "في", "من", "على", "إلى", "عن", "فيه", "عليه", "هو", "هي"]
    text = " ".join([word for word in text.split() if word not in stop_words])

    # Remove unwanted characters
    import re
    text = re.sub(r"\bhttp\S+\b", "", text)
    text = re.sub(r"[^\w\s]", "", text)

    # Apply AraBERT preprocess
    tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
    tokens = tokenizer.tokenize(text)
    processed_text = " ".join(tokens)

    # Apply letter normalization
    text = strip_tashkeel(text)
    text = strip_tatweel(text)
    # Remove English letters
    text = re.sub(r"[a-zA-Z]", "", text)

    return text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00


In [6]:
df['paragraph'] = df['paragraph'].apply(preprocess_arabic_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (563 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [7]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["paragraph"])

# تحويل النصوص إلى تمثيل رقمي
sequences = tokenizer.texts_to_sequences(df["paragraph"])

# تحديد حجم السياق المستخدم في التلخيص
context_size = 100

# إنشاء السياقات والملخصات
contexts = []
summaries = []
for seq in sequences:
    for i in range(context_size, len(seq)):
        context = seq[i-context_size:i]
        summary = seq[i]
        contexts.append(context)
        summaries.append(summary)

# تحويل السياقات والملخصات إلى مصفوفات NumPy
X = np.array(contexts)
y = np.array(summaries)

# تطبيق التعبئة والتغليف للسياقات والملخصات
X = pad_sequences(X, maxlen=context_size)
y = np.expand_dims(y, axis=1)

In [10]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=context_size))
model.add(LSTM(128))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# تجميع النموذج
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# طباعة ملخص لهيكل النموذج
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1444200   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 14442)             1863018   
                                                                 
Total params: 3,424,466
Trainable params: 3,424,466
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
221/221 [==============================] - 54s 195ms/step - loss: 9.0763
Epoch 2/10
221/221 [==============================] - 31s 140ms/step - loss: 8.4992
Epoch 3/10
221/221 [==============================] - 21s 92ms/step - loss: 8.3480
Epoch 4/10
221/221 [==============================] - 14s 63ms/step - loss: 8.1960
Epoch 5/10
221/221 [==============================] - 12s 53ms/step - loss: 8.0047
Epoch 6/10
221/221 [==============================] - 8s 35ms/step - loss: 7.7716
Epoch 7/10
221/221 [==============================] - 8s 39ms/step - loss: 7.5211
Epoch 8/10
221/221 [==============================] - 7s 32ms/step - loss: 7.2769
Epoch 9/10
221/221 [==============================] - 6s 26ms/step - loss: 7.0365
Epoch 10/10
221/221 [==============================] - 5s 21ms/step - loss: 6.8002


In [12]:
!pip install rouge

In [13]:
from rouge import Rouge

rouge = Rouge()

In [18]:
predicted_probabilities = model.predict(X)
generated_summaries = predicted_probabilities.argmax(axis=1)
reference_summaries = y

881/881 [==============================] - 2s 3ms/step


In [ ]:
if isinstance(generated_summaries, str):
    generated_summary_text = generated_summaries
else:
    generated_summary_text = ' '.join([tokenizer.index_word.get(idx.item(), '') for idx in generated_summaries])

if isinstance(reference_summaries, str):
    reference_summary_text = reference_summaries
else:
    reference_summary_text = ' '.join([tokenizer.index_word.get(idx.item(), '') for idx in reference_summaries])

# حساب مقاييس ROUGE
scores = rouge.get_scores(generated_summary_text, reference_summary_text)